**DEEP LEARNING**

Transfer Learning VGG19 on Dog Vs Cat

GABRIEL JOSHUA . R

Dataset: https://www.kaggle.com/datasets/karakaggle/kaggle-cat-vs-dog-dataset

### Importing the dependencies

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers, models

### Installing Kaggle API to download the dataset

In [ ]:
!pip install -q kaggle

### Creating a folder and copying the Kaggle API key to access the dataset


In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

### Downloading the cat vs. dog dataset from Kaggle

In [ ]:
!kaggle datasets download -d karakaggle/kaggle-cat-vs-dog-dataset

### Unzipping the downloaded dataset

In [ ]:
!unzip /content/kaggle-cat-vs-dog-dataset.zip

### Installing the 'split-folders' library for data splitting


In [ ]:
!pip install split-folders

### Splitting the dataset into train, validation, and test sets

In [ ]:
import splitfolders
splitfolders.ratio('/content/kagglecatsanddogs_3367a/PetImages', output="data", seed=1337, ratio=(.7, 0.2, 0.1))

### Data augmentation and preprocessing for training, validation, and test data


In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    '/content/data/train',
    target_size=(256, 256),
    batch_size=32
)

val_data = val_datagen.flow_from_directory(
    '/content/data/val',
    target_size=(256, 256),
    batch_size=32
)

test_data = val_datagen.flow_from_directory(
    '/content/data/test',
    target_size=(256, 256),
    batch_size=32
)


### Get the number of classes and samples for training and validation


In [ ]:
num_classes = train_data.num_classes
nb_train_samples = train_data.samples
nb_val_samples = val_data.samples

### VGG19 model with fine-tuning

In [ ]:
ef VGG19_model_finetuning(freeze_baselayers):
    base_model = tf.keras.applications.vgg19.VGG19(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(256, 256, 3)
    )

    if freeze_baselayers:
        base_model.trainable = False

    input_layer = base_model.inputs
    x = base_model.output
    x = tf.keras.layers.Flatten()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(64, activation="relu")(x)
    output_layer = Dense(2, activation='softmax')(x)

    new_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    return new_model


### Create the VGG19 model with base layers frozen

In [ ]:
model = VGG19_model_finetuning(freeze_baselayers=True)

### Model summary

In [ ]:
model.summary()

### Compile the model with Adam optimizer and categorical cross-entropy loss


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

### Train the model using the training data and validate using the validation data


In [ ]:
history = model.fit_generator(
    train_data,
    steps_per_epoch=nb_train_samples // 32,
    epochs=10,
    validation_data=val_data,
    validation_steps=nb_val_samples // 32
)

### Plotting the training and validation accuracy

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epochs")
plt.legend(["train", "test"], loc="upper left")
plt.show()


### Plotting the training and validation loss

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("loss")
plt.xlabel("epochs")
plt.legend(["train", "test"], loc="upper left")
plt.show()


### Evaluate the model on the test data

In [ ]:
results = model.evaluate(test_data, verbose=0)

print("Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))